# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('walmart').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1617301767328_0008,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [2]:
df=spark.read.csv('walmart_stock.csv',header=True)

#### What are the column names?

In [3]:
df.schema.names

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [4]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)

#### Print out the first 5 columns.

In [5]:
col_name = [df.schema.names[0:5]]
print(col_name)

[['Date', 'Open', 'High', 'Low', 'Close']]

#### Use describe() to learn about the DataFrame.

In [6]:
describe = df.describe().show()
print(describe)

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|         10010500|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [164]:
from pyspark.sql.functions import round, col

describe = df.describe()

describe.select('Date', 
                round('Open', 2).alias('Open'),
                round('High', 2).alias('High'),
                round('Low', 2).alias('Low'),
                round('Close', 2).alias('Close'),
                round('High', 2).alias('High'),
                round('High', 2).alias('High'),
               ).show()

+----------+------+------+
|      Date|  Open|  High|
+----------+------+------+
|      1258|1258.0|1258.0|
|      null| 72.36| 72.84|
|      null|  6.77|  6.77|
|2012-01-03| 56.39| 57.06|
|2016-12-30|  90.8| 90.97|
+----------+------+------+

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [19]:
HVRatio = df["High", "Volume"]
HVRatio = HVRatio.withColumn("HVRatio", df.High/df.Volume)
print(HVRatio.schema.names)
print(HVRatio.head(5))

['High', 'Volume', 'HVRatio']
[Row(High='61.060001', Volume='12668800', HVRatio=4.819714653321546e-06), Row(High='60.349998', Volume='9593300', HVRatio=6.290848613094555e-06), Row(High='59.619999', Volume='12768200', HVRatio=4.669412994783916e-06), Row(High='59.450001', Volume='8069400', HVRatio=7.367338463826307e-06), Row(High='59.549999', Volume='6679300', HVRatio=8.915604778943901e-06)]

#### What day had the Peak High in Price?

In [25]:
from pyspark.sql.functions import max
# peak = HVRatio.select(max["High"]).show()
peak = HVRatio.agg({"High": "max"}).collect()[0][0]
print(peak)

90.970001

#### What is the mean of the Close column?

In [26]:
mean = df.agg({"close": "mean"}).collect()[0][0]
print(mean)

72.38844998012726

#### What is the max and min of the Volume column?

In [28]:
max_volume = df.agg({"Volume": "max"}).collect()[0][0]
print(max_volume)

9994400

In [29]:
min_volume = df.agg({"Volume": "min"}).collect()[0][0]
print(min_volume)

10010500

#### How many days was the Close lower than 60 dollars?

In [38]:
from pyspark.sql.functions import col,when,count

lower_close = df.agg(
    count(when(col("Close") < 60, True)).alias("Less_Than_60")
).show()

#print(lower_close)

+------------+
|Less_Than_60|
+------------+
|          81|
+------------+

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [54]:
higher_80_close = df.agg(count(when(col("High") > 80, True))).collect()[0][0]
total_days = df.count()
# print(higher_80_close)
print("{:.0%}".format(higher_80_close/total_days))


8%

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [80]:
df = df.withColumn("High", df.High.cast('integer'))
df = df.withColumn("Volume", df.Volume.cast('int'))
pearson_correlation = df.stat.corr("High", "Volume")
print(pearson_correlation)

-0.33761286268813967

#### What is the max High per year?

In [92]:
from pyspark.sql.functions import year
max_high_year = df.groupBy(year("Date").alias('Year')).agg({"High": "max"}).orderBy('Year').show()
#print(max_high_year)


+----+---------+
|Year|max(High)|
+----+---------+
|2012|       77|
|2013|       81|
|2014|       88|
|2015|       90|
|2016|       75|
+----+---------+

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [95]:
from pyspark.sql.functions import month
avg_calendar_close_price = df.groupBy(month("date").alias('Month')).agg({"Close": "mean"}).orderBy('Month').show()


+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

# Great Job!